In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.svm import SVC,NuSVC,OneClassSVM
from sklearn import svm, datasets
from sklearn.kernel_approximation import Nystroem,RBFSampler
from sklearn.metrics import roc_auc_score

In [ ]:
total_data=pd.read_csv('income2023f/totaldata.csv')
total_data = total_data.iloc[: , 1:]
train_datam=total_data[total_data['income>50K']!=-999]
test_datam=total_data[total_data['income>50K']==-999]
train_datam.loc[train_datam['income>50K']==0,['income>50K']]=-1
train_data=train_datam.iloc[:20000]
test_data=train_datam.iloc[20000:]
test_data=test_data.reset_index(drop=True)

<ipython-input-4-9f08b7fcda67>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_datam.loc[train_datam['income>50K']==0,['income>50K']]=-1


In [ ]:
total_data

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income>50K
0,53,1,93449,1,15,1,1,1,1,1,0,0,40,1,1
1,33,1,123424,2,13,1,2,1,2,1,0,0,40,2,1
2,47,2,144844,3,9,1,3,1,2,1,0,0,40,2,0
3,40,2,114580,3,9,2,3,2,2,2,0,0,40,3,0
4,39,2,115618,3,9,1,4,1,2,1,0,0,50,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,26,2,43408,2,13,1,1,1,2,1,0,0,40,2,-999
48838,27,2,116372,2,13,3,1,4,2,2,0,0,40,2,-999
48839,67,5,182581,6,10,1,2,1,2,1,20051,0,20,2,-999
48840,46,4,274689,10,12,2,13,4,2,1,0,0,40,2,-999


In [ ]:
total_data_encoded=pd.get_dummies(total_data, columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country'])

In [ ]:
total_data_encoded #one hot encoding

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income>50K,workclass_1,workclass_2,workclass_3,...,native.country_32,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41
0,53,93449,15,0,0,40,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,123424,13,0,0,40,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,47,144844,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,40,114580,9,0,0,40,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,39,115618,9,0,0,50,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,26,43408,13,0,0,40,-999,0,1,0,...,0,0,0,0,0,0,0,0,0,0
48838,27,116372,13,0,0,40,-999,0,1,0,...,0,0,0,0,0,0,0,0,0,0
48839,67,182581,10,20051,0,20,-999,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48840,46,274689,12,0,0,40,-999,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
column_to_move = 'income>50K'
total_data_encoded[column_to_move] = total_data_encoded.pop(column_to_move)


In [ ]:
total_data_encoded

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_1,workclass_2,workclass_3,workclass_4,...,native.country_33,native.country_34,native.country_35,native.country_36,native.country_37,native.country_38,native.country_39,native.country_40,native.country_41,income>50K
0,53,93449,15,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,33,123424,13,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,47,144844,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,114580,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,39,115618,9,0,0,50,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,26,43408,13,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-999
48838,27,116372,13,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-999
48839,67,182581,10,20051,0,20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-999
48840,46,274689,12,0,0,40,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-999


In [ ]:
total_data_encoded=total_data_encoded.drop('fnlwgt',axis=1) #removing the column to see if results change

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
total_data_encoded.iloc[:,:-1]=scaler.fit_transform(total_data_encoded.iloc[:,:-1]) #normalizing

In [ ]:
train_datam=total_data_encoded[total_data_encoded['income>50K']!=-999]
test_datam=total_data_encoded[total_data_encoded['income>50K']==-999]
train_datam.loc[train_datam['income>50K']==0,['income>50K']]=-1
train_data=train_datam.iloc[:20000]
test_data=train_datam.iloc[20000:]
test_data=test_data.reset_index(drop=True)

In [ ]:
y=train_data.iloc[:,-1]
x=train_data.iloc[:,:-1]
x_train=x.to_numpy()
y_train=y.to_numpy()

In [ ]:
y=train_datam.iloc[:,-1]
x=train_datam.iloc[:,:-1]
x_trainm=x.to_numpy()
y_trainm=y.to_numpy()

In [ ]:
y=test_data.iloc[:,-1]
x=test_data.iloc[:,:-1]
x_test=x.to_numpy()
y_test=y.to_numpy()

In [ ]:
y=test_datam.iloc[:,-1]
x=test_datam.iloc[:,:-1]
x_testm=x.to_numpy()
y_testm=y.to_numpy()

In [ ]:
# splitting of total data to the required training and test splits

(15174,)

In [ ]:
svm2=SVC(C=0.5,gamma=0.009) #  svm
svm2.fit(x_train,y_train)

SVC(C=0.5, gamma=0.009)

In [ ]:
lsvm=svm.LinearSVC()
feat_data=RBFSampler(gamma='scale',n_components=10000,random_state=98) #inserting non linearity
data_tran=feat_data.fit_transform(x_train)
lsvm.fit(data_tran,y_train)

LinearSVC()

In [ ]:
data_tran=feat_data.fit_transform(x_train)
g=lsvm.predict(data_tran)

In [ ]:
svm1 = NuSVC(nu=0.26,kernel='rbf',gamma=0.007,class_weight='balanced')
svm1.fit(x_train, y_train)

NuSVC(class_weight='balanced', gamma=0.007, nu=0.26)

In [ ]:
nsvm=OneClassSVM(nu=0.0001,gamma=0.001)
nsvm.fit(x_train_n)

OneClassSVM(gamma=0.001, nu=0.0001)

In [ ]:
h=nsvm.predict(x_test)
h[h==-1]=0
h[h==1]=-1
h[h==0]=1
res=h*y_test
res[res==-1]=0
sum(res)/res.shape[0]

0.8022

In [ ]:
gamma=[0.009,0.007,0.0065,0.01]
nu=[0.1,0.25,0.3,0.32]
for i in gamma:
  for j in nu:
    svm1 = NuSVC(nu=0.26,kernel='rbf',gamma=0.007)
    svm1.fit(x_train, y_train)

In [ ]:
g=svm1.predict(x_train)

In [ ]:
res=g*y_train

In [ ]:
res[res==-1]=0

In [ ]:
sum(res)/res.shape[0]

0.8936

In [ ]:
h=svm1.predict(x_test)
res=h*y_test
res[res==-1]=0
sum(res)/res.shape[0]

0.8672

In [ ]:
import csv
a=list(range(1,23843))
with open('/content/drive/MyDrive/income2023f/nusvc.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(a,h))

In [ ]:
h=svm2.predict(x_test)
res=h*y_test
res[res==-1]=0
sum(res)/res.shape[0]

In [ ]:
import csv
a=list(range(1,23843))
with open('/content/drive/MyDrive/income2023f/svc.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(a,h))

In [ ]:
roc_auc_score(y_train, svm1.decision_function(x_train))

0.9387554840436105

In [ ]:
roc_auc_score(y_test, svm1.decision_function(x_test))

0.8868816030349148

In [ ]:
data_tran=feat_data.fit_transform(x_test)

In [ ]:
h=nsvm.predict(x_testm)
h[h==-1]=0
h[h==1]=-1
h[h==0]=1

In [ ]:
#CSV writer from 'h' - the results
import csv
a=list(range(1,23843))
with open('/content/drive/MyDrive/income2023f/onesvc.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(a,h))